In [ ]:
import flavio
import flavio.plots
import wcxf
import numpy as np
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

# flavio tutorial

## Part 4: Fits

## "Fits" in flavio

A fit is defined as a selection of *measurements* and *observables* providing a likelihood in the space of a selection of *parameters* or *Wilson coefficients*

Examples:

- Fit the Wilson coefficient $C_9^{bs\mu\mu}$ from the LHCb measurement of $R_K$
- Fit the CKM element $V_{cb}$ from $B$ factory measurements of $\text{BR}(B\to D\ell\nu)$

### Ingredients for a fit

- a list of observables for which measurements exist
- a list of parameters or Wilson coefficients to be fitted
- a set of constraints on the parameters (such as `flavio.default_parameters`) summarizing our prior knowledge

## Statistical inference: the likelihood

- Observables $x$, Measurements $x_i^\text{exp}$, Predictions $x^\text{th}$, Parameters $\theta$

$$\mathcal L_\text{exp}(\vec\theta)=\prod_{i=1}^N f_i\!\left(x_i^\text{exp}, x^\text{th}_i(\vec\theta)\right)$$

Challenge: we are only interested in some of the $\theta_j$ ("fit parameters") but must take into account the uncertainties induced be the others ("nuisance parameters")

Three statistical approaches to this problem implemented in flavio:

#### "Fast fit"

convolve the experimental uncertainties with the theoretical ones to construct a likelihood that only depends on fit parameters

$$\mathcal L =e^{-\chi^2(\vec\xi)/2},
\qquad\chi^2(\vec\xi)=\vec\Delta^T C^{-1}(\vec\xi=\hat{\vec\xi}) \vec\Delta$$
$$\Delta_i = (x_i^\text{exp}- x^\text{th}_i(\vec\theta),
\qquad C(\vec\xi)=C_\text{exp}+C_\text{th}(\vec\xi)$$

#### Bayesian fit

treat the uncertainties on nuisance parameters as prior probabilities and use Bayes' theorem to *marginalize* over them (multi-dimensional integration requiring tools like Markov Chain Monte Carlo)


$$P(\vec\theta) \propto \mathcal L_\text{exp}(\vec\theta)\pi(\vec\theta)$$

$$P(\vec\xi)=\int d\vec\nu ~ P(\vec\xi,\vec\nu)$$

#### Frequentist fit

treat the uncertainties on nuisance parameters as pseudo measurements and *profile* over them, i.e. optimizing the likelihood in nuisance parameter space for each point in fit parameter space

$$\mathcal L_\text{tot}(\vec\theta) = \mathcal L_\text{exp}(\vec\theta) \mathcal L_\text{nuis}(\vec\nu)$$

$$\mathcal L_\text{p}(\vec\xi) = \mathcal L_\text{tot}(\vec\xi, \hat{\hat{\vec\nu}})$$

### Choosing the fit type: advantages & disadvantages


### Fast fit

Assumptions & advantages:
- (!) experimental uncertainties approximated as Gaussian
- (!) theory uncertainties approximated as Gaussian
- (!) uncertainties weakly dependent on nuisances and NP
- (+) Computing time does not depend on number of nuisance parameters
- (+) Computation of covariances (expensive) does not depend on exp. data

### Bayesian fit

- (-) Need to perform MCMC with at least $O(100\,\text{k})$ steps (evaluations of the likelihood)
- (+) No distinction between nuisance and fit parameters: can obtain posterior for all of them

### Frequentist fit

- (-) Optimization on grid in fit parameter space limits number of fit parameters to 1 or 2 in practice
- (+) Not subject to statistical fluctuations from random sampling
- (+) Faster than Bayesian MCMC for 1 or 2 fit parameters

## Setting up a "Fast Fit"

Example: $V_{cb}$ from $\text{BR}(B^+\to D^0\ell\nu)$, $\ell=e$ or $\mu$

First try:

In [ ]:
from flavio.statistics import fits
fit = fits.FastFit(name="My First Fast Fit",
                   fit_parameters=['Vcb'],
                   observables=['BR(B+->Denu)', 'BR(B+->Dmunu)'])

Let's have a look at the Prior likelihood for `Vcb`

In [ ]:
import numpy as np

x = np.linspace(0.03, 0.05, 500)
y = [fit.log_prior_parameters([X]) for X in x]

flavio.plots.likelihood_plot(x, np.exp(y))

What went wrong?

The fast fit takes the priors for nuisance *and* fit parameters from a `ParameterConstraints` instance - by default, `flavio.default_parmeters`, which also contains a constraint on $V_{cb}$.

Solution: replace by a "flat" prior (range)

In [ ]:
my_parameters = flavio.default_parameters.copy()
my_parameters.set_constraint('Vcb', '[0.03, 0.05]')

fit = fits.FastFit(name="My First Fast Fit",
                   par_obj=my_parameters,
                   fit_parameters=['Vcb'],
                   observables=['BR(B+->Denu)', 'BR(B+->Dmunu)'])

In [ ]:
x = np.linspace(0.025, 0.055, 100)
y = [fit.log_prior_parameters([X]) for X in x]

flavio.plots.likelihood_plot(x, np.exp(y))

Better!

### A word of caution

This is a general pitfall: some constraints on parameters actually come from measurements of observables that are present in flavio as well

### Computing the covariance

To compute the covariance of the "pseudo measurement" with combined experimental and theoretical uncertainties, need to call:

In [ ]:
%time fit.make_measurement(N=100, force=True)  # force recomputation for demo

Advanced: increase precision, use parallelization

In [ ]:
%time fit.make_measurement(N=1000, threads=2, force=True)

We're done: evaluate the likelihood, which only depends on `Vcb`

In [ ]:
x = np.linspace(0.03, 0.05, 100)
y = [fit.log_likelihood([X]) for X in x]

flavio.plots.likelihood_plot(x, np.exp(y))

Best-fit value for $V_{cb}$:

In [ ]:
bf = fit.best_fit()
bf

Determining uncertainties: numerically solve for $\Delta \chi^2=1$

In [ ]:
from scipy.optimize import brentq

err_l = brentq(lambda x: -2*fit.log_likelihood([x]) - 1, a=0.03, b=bf['x'])
err_r = brentq(lambda x: -2*fit.log_likelihood([x]) - 1, a=bf['x'], b=0.05)

err_l, err_r

Final result: $V_{cb} = ( 3.75 ^{+0.04}_{-0.03} ) \times 10^{-2}$

### Example: $C_7$ and $C_{10}$ from $B\to X_s\gamma$ and $B_s\to\mu^+\mu^-$


Fitting Wilson coefficients: since quantities to be fitted must be real but WCs are complex, must always define a WC *function*, e.g.

In [ ]:
def fct_C7_C10(Re_C7, Re_C10):
    return {'C10_bsmumu': Re_C10, 'C7_bs': Re_C7}

Output must be in a form suitable for feeding it to `WilsonCoefficients.set_initial`.

Defining the fit

In [ ]:
fit = fits.FastFit(name='My C7-C10 fast fit',
                   observables=['BR(B->Xsgamma)', 'BR(Bs->mumu)', 'BR(B0->mumu)'],
                   fit_wc_function=fct_C7_C10,
                   input_scale=4.8)

NB: `BR(B0->mumu)` is not sensitive to these coefficients but must be added since the measurements are correlated. Try running the command without it!

Generate the pseudo measurement

In [ ]:
%time fit.make_measurement(N=100, threads=2, force=True)

Get best-fit values

In [ ]:
fit.best_fit()

In [ ]:
fit.fit_parameters, fit.fit_wc_names

## 2D likelihood plots

In [ ]:
import flavio.plots as fpl
import matplotlib.pyplot as plt

In [ ]:
%%time
cdat = fpl.likelihood_contour_data(fit.log_likelihood,
                                   x_min=-0.07, x_max=0.07, y_min=-2, y_max=2,
                                   n_sigma=(1, 2, 3),
                                   threads=2,
                                   steps=20)

In [ ]:
fpl.contour(**cdat)
plt.xlabel(r'Re $C_7$');
plt.ylabel(r'Re $C_{10}$');

This can be made more smooth by increasing `steps` or using interpolation (but be careful...)

In [ ]:
fpl.contour(**cdat, interpolation_factor=3)
plt.xlabel(r'Re $C_7$');
plt.ylabel(r'Re $C_{10}$');

## Bayesian fit

The definition is analogous to the Fast Fit. But we now need to explicitly specify the nuisance parameters we want to be varied (since we now pay computing time for each of them!)

### Finding out which parameters to vary

A useful hint (but not always sufficient!) can be to look at the parameters the SM prediction depends on:

In [ ]:
flavio.functions.get_dependent_parameters_sm('BR(Bs->mumu)')

For now, let's use

In [ ]:
nuisance_parameters = ['Vcb', 'delta_BXsgamma', 'm_t', 'f_Bs']

### Additional complication

- even if we usually want to treat our Wilson coefficients as unconstrained (i.e. flat priors), we need to provide some initial ranges for the samplers to generate reasonable starting values

In [ ]:
star_wc_priors = flavio.classes.WilsonCoefficientPriors()
star_wc_priors.set_constraint('Re_C7', '[-0.2, 0.2]')
star_wc_priors.set_constraint('Re_C10', '[-4, 4]')

Define the fit instance

In [ ]:
fit = fits.BayesianFit(name='My C7-C10 Bayesian fit',
                       observables=['BR(B->Xsgamma)', 'BR(Bs->mumu)', 'BR(B0->mumu)'],
                       nuisance_parameters=nuisance_parameters,
                       fit_wc_function=fct_C7_C10,
                       input_scale=4.8,
                       start_wc_priors=star_wc_priors)

### Marginalizing over the nuisance parameters

The `flavio.statistics.fitters` module contains interfaces to 2 MCMC samplers: `pypmc` and `emcee`.

Let's use the latter, with default settings

In [ ]:
from flavio.statistics.fitters.emcee import emceeScan

scan = emceeScan(fit)

In [ ]:
%time scan.run(steps=50, burnin=50)

In [ ]:
scan.result.shape

NB, this result does not make any sense as we need much more statistics to get a reliable answer!

Plotting the result

## Frequentist fits

Again we need to specify the nuisance parameters; we can use the same as in the Bayesian fit

In [ ]:
fit = fits.FrequentistFit(name='My C7-C10 frequentist fit',
                          observables=['BR(B->Xsgamma)', 'BR(Bs->mumu)', 'BR(B0->mumu)'],
                          nuisance_parameters=nuisance_parameters,
                          fit_wc_function=fct_C7_C10,
                          input_scale=4.8)

### Likelihood profiling

In contrast to the Bayesian case, we are now limited to 1-2 fit parameters or Wilson coefficients. The likelihood profiling has to be performed on a grid

In [ ]:
from flavio.statistics.fitters.profiler import Profiler2D

prof = Profiler2D(fit, x_min=-0.07, x_max=0.07, y_min=-2, y_max=2)

In [ ]:
%time prof.run(steps=(10, 10), usebf=True);

In [ ]:
pdat = prof.contour_plotdata(n_sigma=(1, 2, 3))
fpl.contour(**pdat, interpolation_factor=3)
plt.xlabel(r'Re $C_7$');
plt.ylabel(r'Re $C_{10}$');

## Problem:

Reproduce the fits of $V_{cb}$ from global fits to $B\to D\ell\nu$ and $B\to D^*\ell\nu$ in eqs. (7)-(9), (12)-(14) of [arXiv:1801.01112](https://arxiv.org/pdf/1801.01112.pdf), as well as figure 1.

The list of observables and nuisance is provided in the file `vcbfits.py`

Next: [Extending flavio](5 Extending.ipynb)